# Prove

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from pytorch_tabular import TabularModel
from pytorch_tabular.models import TabNetModelConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
    ExperimentConfig,
)
import torch

FILENAME = "train.csv"
df = pd.read_csv(FILENAME)

X = df.drop("Year", axis=1)
y = df["Year"]

# Esegui lo split in set di addestramento, di test e di validazione
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, random_state=42, test_size=0.2)
train, val = train_test_split(train, random_state=42, test_size=0.2)

# Configurazione del modello TabNet per la regressione
data_config = DataConfig(
    target=["Year"],
    continuous_cols=X.columns.tolist(),
)

optimizer_config = OptimizerConfig()  # Imposta manualmente il tasso di apprendimento
model_config = TabNetModelConfig(
    task="regression",
    n_d=16,       # Riduci il numero di decisioni
    n_a=16,       # Riduci il numero di attenzioni
    n_steps=3,    # Riduci il numero di passaggi
    gamma=1.0,    # Modifica il valore di gamma
    metrics=['mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error', 'r2_score'],
)

trainer_config = TrainerConfig(
    batch_size=256,
    auto_lr_find=False,  # Disabilita la ricerca automatica del tasso di apprendimento
    max_epochs=1,
    early_stopping_patience=10,
)

# Creazione e addestramento del modello TabNet
tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

# Aumenta il numero di workers nei DataLoader
tabular_model.fit(train=train, validation=val)

torch.save(tabular_model, open("tabNet.save", 'wb'))

result = tabular_model.evaluate(test)
pred_df = tabular_model.predict(test)


/Users/alessandrotocco/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


2024-02-11 23:25:13,926 - {pytorch_tabular.tabular_model:140} - INFO - Experiment Tracking is turned off

Seed set to 42


2024-02-11 23:25:14,077 - {pytorch_tabular.tabular_model:524} - INFO - Preparing the DataLoaders

2024-02-11 23:25:14,153 - {pytorch_tabular.tabular_datamodule:499} - INFO - Setting up the datamodule for          
regression task

...

2024-02-11 23:26:26,345 - {pytorch_tabular.tabular_model:663} - INFO - Training the model completed

2024-02-11 23:26:26,349 - {pytorch_tabular.tabular_model:1489} - INFO - Loading the best model

Output()

/Users/alessandrotocco/Library/Python/3.9/lib/python/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │         3746473.5         │
│  test_mean_squared_error  │         3746473.5         │
└───────────────────────────┴───────────────────────────┘